In [5]:
# !pip install selenium
# !pip install scrapy
# !pip install typing-extensions==4.3.0

In [6]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
import os
warnings.filterwarnings("ignore")

In [7]:
filepath = open('../data/raw/urls/imdb.txt', 'r')
lines = filepath.readlines()

In [8]:
for url in lines:
    url = url+'/reviews'
    driver = webdriver.Chrome()
    time.sleep(1)
    driver.get(url)
    # time.sleep(1)
    movie_name = driver.title.split(')')[0]
    print(movie_name)
    if movie_name in os.listdir('../data/raw/imdb_reviews/'):
        pass
    # time.sleep(1)

    body = driver.find_element(By.CSS_SELECTOR, 'body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    body.send_keys(Keys.PAGE_DOWN)

    sel = Selector(text = driver.page_source)
    review_counts = sel.css('.lister .header span::text').extract_first().replace(',','').split(' ')[0]
    print(review_counts)
    more_review_pages = int(int(review_counts)/25)

    for i in tqdm(range(more_review_pages)):
        try:
            # body.send_keys(Keys.PAGE_DOWN)
            # time.sleep(1)
            # body.send_keys(Keys.PAGE_DOWN)
            # time.sleep(1)
            # body.send_keys(Keys.PAGE_DOWN)
            # time.sleep(1)
            # body.send_keys(Keys.PAGE_DOWN)
            # time.sleep(1)
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            time.sleep(3)
            css_selector = 'load-more-trigger'
            driver.find_element(By.ID, css_selector).click()
            # time.sleep(0.5)
        except:
            pass
    
    # review = sel2.css('.text.show-more__control::text').extract_first().strip()
    # review_date = sel2.css('.review-date::text').extract_first().strip()
    # author = sel2.css('.display-name-link a::text').extract_first().strip()
    # review_title = sel2.css('a.title::text').extract_first().strip()
    # review_url = sel2.css('a.title::attr(href)').extract_first().strip()
    # helpfulness = sel2.css('.actions.text-muted::text').extract_first().strip()
    # print('nRating:',rating)
    # print('nreview_title:',review_title)
    # print('nAuthor:',author)
    # print('nreview_date:',review_date)
    # print('nreview:',review)
    # print('nhelpfulness:',helpfulness)


    rating_list = []
    review_date_list = []
    review_title_list = []
    author_list = []
    review_list = []
    review_url_list = []
    error_url_list = []
    error_msg_list = []
    reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')

    for d in tqdm(reviews):
        try:
            sel2 = Selector(text = d.get_attribute('innerHTML'))
            try:
                rating = sel2.css('.rating-other-user-rating span::text').extract_first()
            except:
                rating = np.NaN
            try:
                review = sel2.css('.text.show-more__control::text').extract_first()
            except:
                review = np.NaN
            try:
                review_date = sel2.css('.review-date::text').extract_first()
            except:
                review_date = np.NaN    
            try:
                author = sel2.css('.display-name-link a::text').extract_first()
            except:
                author = np.NaN    
            try:
                review_title = sel2.css('a.title::text').extract_first()
            except:
                review_title = np.NaN
            try:
                review_url = sel2.css('a.title::attr(href)').extract_first()
            except:
                review_url = np.NaN
            rating_list.append(rating)
            review_date_list.append(review_date)
            review_title_list.append(review_title)
            author_list.append(author)
            review_list.append(review)
            review_url_list.append(review_url)
        except Exception as e:
            error_url_list.append(url)
            error_msg_list.append(e)
    movie = [movie_name for i in range(len(review_date_list))]
    review_df = pd.DataFrame({
        'Review_Date':review_date_list,
        'Author':author_list,
        'Rating':rating_list,
        'Review_Title':review_title_list,
        'Review':review_list,
        'Review_Url':review_url,
        'Movie' : movie
        })
    review_df.to_csv(f'../data/raw/imdb_reviews/{movie_name}.csv')
    # break